In [1]:
!pip install requests beautifulsoup4 psycopg2-binary



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import psycopg2

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       # Ganti dengan nama database Anda
    user="postgres",         # Ganti dengan username PostgreSQL Anda
    password="felixexe131105",     # Ganti dengan password PostgreSQL Anda
    host="localhost",         # Jika PostgreSQL berjalan di komputer lokal
    port="5432"                   # Port default PostgreSQL
)

cursor = connection.cursor()
print("Koneksi berhasil!")


Koneksi berhasil!


In [2]:
import psycopg2
import requests
from bs4 import BeautifulSoup

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       
    user="postgres",             
    password="felixexe131105",   
    host="localhost",            # Ganti dengan host jika berbeda
    port="5432"                  # Port default PostgreSQL
)

cursor = connection.cursor()

# URL dari halaman e-commerce (contoh halaman produk)
url = "https://www.tokopedia.com/search?st=&q=yamaha%20&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="

# Lakukan HTTP GET request
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

# Parse konten halaman menggunakan BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Cari elemen HTML yang berisi nama dan harga produk
produk = []
for item in soup.find_all("div", class_="_6+OpBPVGAgqnmycna+bWIw=="):  # Sesuaikan selector berdasarkan situs
    nama = item.find("span", class_="_0T8-iGxMpV6NEsYEhwkqEg==").get_text()  # Selector untuk nama produk
    
    # Cari harga produk
    harga_item = item.find_next("div", class_="_67d6E1xDKIzw+i2D2L0tjw== t4jWW3NandT5hvCFAiotYg==")
    if harga_item:
        harga = harga_item.get_text().strip()  # Ambil harga dan hilangkan spasi
        harga = int(harga.replace("Rp", "").replace(".", ""))  # Konversi harga ke integer (hapus "Rp" dan titik)
        produk.append((nama, harga))

# Masukkan data ke tabel ProdukEcommerce
for p in produk:
    try:
        # Query untuk memasukkan data ke dalam tabel
        query = "INSERT INTO ProdukEcommerce (nama_produk, harga) VALUES (%s, %s)"
        cursor.execute(query, (p[0], p[1]))  # Eksekusi query dengan data nama dan harga produk
        print(f"Data berhasil dimasukkan: {p[0]} | Harga: {p[1]}")
    except Exception as e:
        print(f"Error saat memasukkan data: {e}")

# Commit perubahan ke database
connection.commit()

# Tutup koneksi
cursor.close()
connection.close()


Data berhasil dimasukkan: 2024 - Yamaha All New XSR 155 | Harga: 25900000
Data berhasil dimasukkan: Yamaha Lexi LX Standart 155 Sepeda Motor Matic | Harga: 25900000
Data berhasil dimasukkan: 2024 - Yamaha Nmax Neo Version | Harga: 125000
Data berhasil dimasukkan: caferacer cover jok kulit jok sadel honda yamaha srung jok kulit jok | Harga: 125000
Error saat memasukkan data: value too long for type character varying(100)

Error saat memasukkan data: current transaction is aborted, commands ignored until end of transaction block

Error saat memasukkan data: current transaction is aborted, commands ignored until end of transaction block

Error saat memasukkan data: current transaction is aborted, commands ignored until end of transaction block

Error saat memasukkan data: current transaction is aborted, commands ignored until end of transaction block

Error saat memasukkan data: current transaction is aborted, commands ignored until end of transaction block



In [11]:
!pip install requests-html



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
pip install lxml[html_clean]


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\felix\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import psycopg2
import time

# Koneksi ke PostgreSQL
connection = psycopg2.connect(
    dbname="db_gudang_04",       
    user="postgres",             
    password="felixexe131105",   
    host="localhost",            
    port="5432"                  
)
cursor = connection.cursor()

# URL dari halaman e-commerce
url = "https://www.tokopedia.com/search?st=&q=yamaha%20&srp_component_id=02.01.00.00&srp_page_id=&srp_page_title=&navsource="

# Setup Selenium WebDriver
service = Service('/path/to/chromedriver')  # Sesuaikan path ke chromedriver
driver = webdriver.Chrome(service=service)

# Akses halaman
driver.get(url)
time.sleep(5)  # Tunggu halaman selesai dimuat

# Parse halaman menggunakan BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Scraping data
produk = []
for item in soup.find_all("div", class_="css-1sn1xa2"):
    # Nama produk
    nama_tag = item.find("div", class_="css-1b6t4dn")
    nama_produk = nama_tag.get_text().strip() if nama_tag else None

    # URL produk
    link_tag = item.find("a", href=True)
    url_produk = link_tag["href"] if link_tag else None

    # Gambar produk
    img_tag = item.find("img", class_="success fade")
    img_url = img_tag["src"] if img_tag else None

    # Simpan data jika semua elemen ditemukan
    if nama_produk and url_produk and img_url:
        produk.append((nama_produk, url_produk, img_url))

# Masukkan data ke tabel CrawlingData
for p in produk:
    try:
        query = "INSERT INTO CrawlingData (nama_produk, url_produk, img_url) VALUES (%s, %s, %s)"
        cursor.execute(query, (p[0], p[1], p[2]))
        print(f"Data berhasil dimasukkan: {p[0]} | URL: {p[1]} | Gambar: {p[2]}")
    except Exception as e:
        print(f"Error saat memasukkan data: {e}")

# Commit dan tutup koneksi
connection.commit()
cursor.close()
connection.close()

print("Proses selesai.")


ModuleNotFoundError: No module named 'selenium'